## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [1]:
import torch
import numpy as np
# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([False,  True,  True,  ...,  True,  True,  True])

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [2]:
def get_parallel_db(db, remove_index):
    return torch.cat((db[:remove_index], db[remove_index+1:]))

In [3]:
def get_parallel_dbs(db):
    parallel_dbs = []
    
    for i in range(len(db)):
        parallel_dbs.append(get_parallel_db(db, i))
    return parallel_dbs

In [4]:
def create_db_and_parallels(n):
    '''Function to create an original database of size n containing 0 and 1,
    and n parallel databases, each with one less element compared to original database.
    Return:
    db: original database of size n;
    pdbs: list of n parallel databases
    '''
    db = torch.randint(0,2, size=(n,))
    pdbs = get_parallel_dbs(db)
    return db, pdbs

In [10]:
db, pdbs = create_db_and_parallels(5000)

In [11]:
len(pdbs)

5000

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [12]:
db, pdbs = create_db_and_parallels(5000)

In [13]:
def query(db):
    return db.sum()

In [14]:
full_db_result = query(db)

In [15]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [16]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [7]:
def query(db):
    return torch.mean(db.float())

In [5]:
def sensitivity(query, n_entries):
    db, pdbs = create_db_and_parallels(5000)
    
    full_db_result = query(db)
    
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
    return sensitivity

In [8]:
sensitivity(query, 5000)

tensor(0.0001)

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [22]:
def query(db, threshold):
    return (db.sum() > threshold).float()

In [23]:
def create_dbs(n_entries):
    dbs = list()
    for i in range(n_entries):
        dbs.append(torch.randint(0, 2, size=(n_entries,)))
    return dbs

In [24]:
def sensitivity(query, n_entries):
    dbs = create_dbs(n_entries) # pdbs contains 10 databases of size 10
    
    sensitivities = list()
    for db in dbs:
        sensitivities.append(query(db, threshold=5))
    return sensitivities

In [25]:
sensitivity(query, 10)

[tensor(0.),
 tensor(0.),
 tensor(1.),
 tensor(0.),
 tensor(1.),
 tensor(0.),
 tensor(1.),
 tensor(1.),
 tensor(0.),
 tensor(0.)]

# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [26]:
def query_sum(db):
    return db.sum()

In [43]:
def query_sum_threshold(db, threshold=4):
    return (db.sum() > threshold).float()

In [44]:
def sensitivity(query, n_entries):
    # create a database of size 10
    db = torch.randint(0, 2, size=(n_entries,))
    
    # query on the whole database
    db_result = query(db)
    
    # query on the database without 10th entry
    reduced_db_result = query(db[:-1])
    
    # calculate sensitivity
    return torch.abs(db_result-reduced_db_result)

In [45]:
sensitivity(query_sum, 10)

tensor(1)

In [46]:
sensitivity(query_sum_threshold, 10)

tensor(0.)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [2]:
# create a database
db = torch.randint(0, 2, size=(1000,)) 

In [4]:
def query(database):
    return torch.mean(database.float())

In [6]:
def report_query(n_entries):
    db = torch.randint(0, 2, size=(n_entries,))
    first_flip = torch.randint(0, 2, size=(n_entries,))
    second_flip = torch.randint(0, 2, size=(n_entries,))
    augmented_db = db*first_flip + (1 - first_flip)*second_flip   
                                
    db_result = query(db)
    augmented_db_result = query(augmented_db)*2 - 0.5
    return db_result, augmented_db_result 

In [7]:
print(report_query(10))

(tensor(0.8000), tensor(0.5000))


In [8]:
print(report_query(100))

(tensor(0.5300), tensor(0.6200))


In [9]:
print(report_query(1000))

(tensor(0.5160), tensor(0.5100))


In [10]:
print(report_query(10000))

(tensor(0.5016), tensor(0.5046))


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [11]:
def query(database):
    return torch.mean(database.float())

In [14]:
def report_query(n_entries, noise):
    
    "noise: how likely the first flip will be 1"
    
    db = torch.randint(0, 2, size=(n_entries,))
    
    first_flip = (torch.rand(len(db)) > (1 - noise)).float() # random numbers from a uniform distribution on the interval [0, 1)
    second_flip = (torch.rand(len(db)) > 0.5).float()
    
    augmented_db = db*first_flip + (1 - first_flip)*second_flip   
                                
    db_result = query(db)
    
    augmented_db_result = (query(augmented_db) - 0.5*(1-noise))/noise
    
    return db_result, augmented_db_result 

In [15]:
# if noise = 0.3: 70% the chance the first flip will be 1

In [26]:
report_query(n_entries=100, noise=0.1)

(tensor(0.4400), tensor(0.4000))

In [27]:
report_query(n_entries=100, noise=0.2)

(tensor(0.5000), tensor(0.7500))

In [28]:
report_query(n_entries=100, noise=0.4)

(tensor(0.4900), tensor(0.4500))

In [29]:
report_query(n_entries=100, noise=0.8)

(tensor(0.4900), tensor(0.5125))

In [30]:
report_query(n_entries=100, noise=1)

(tensor(0.5000), tensor(0.5000))

In [34]:
report_query(n_entries=10000, noise=0.1)

(tensor(0.4977), tensor(0.5220))

In [35]:
report_query(n_entries=10000, noise=0.2)

(tensor(0.4999), tensor(0.5200))

In [36]:
report_query(n_entries=10000, noise=0.4)

(tensor(0.5001), tensor(0.4940))

# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [5]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(45.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [7]:
def query_sum(db):
    return torch.sum(db.float())

In [8]:
def query_mean(db):
    return torch.mean(db.float())

In [38]:
db, pdbs = create_db_and_parallels(5000)

In [39]:
def laplacian_mechanism(query, db, sensitivity, epsilon):
    beta = sensitivity/epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    return query(db) + noise

In [40]:
laplacian_mechanism(query_sum, db, 1, 1)

tensor([2492.6993], dtype=torch.float64)

In [41]:
laplacian_mechanism(query_mean, db, 1/5000, 1)

tensor([0.4988], dtype=torch.float64)

In [43]:
# reduce epsilon - noise is more spread out
laplacian_mechanism(query_sum, db, 1, 0.001)

tensor([2326.1991], dtype=torch.float64)

In [44]:
laplacian_mechanism(query_mean, db, 1/5000, 0.001)

tensor([0.3998], dtype=torch.float64)

# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [2]:
import numpy as np

In [3]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [19]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions

In [20]:
preds[0] # predictions for first example from 10 teacher classifiers

array([8, 4, 6, 5, 2, 2, 3, 4, 5, 5])

In [21]:
preds.shape

(10000, 10)

In [22]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1) # add noise to prediction counts

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [25]:
len(new_labels)

10000

# PATE Analysis

In [2]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [59]:
from syft.frameworks.torch.differential_privacy import pate

In [61]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps



In [64]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [65]:
preds[:,0:50] *= 0

In [66]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

### 1. Load MNIST dataset

In [68]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import matplotlib.pyplot as plt

In [75]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))])
teacher_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
student_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [72]:
print("Number of examples in teacher data: " + str(len(teacher_data)))
print("Number of examples in student data: " + str(len(student_data)))

Number of examples in teacher data: 60000
Number of examples in student data: 10000


I will divide teacher dataset into 200 different teacher subsets, each subset contains 300 samples.

I will split each subset into 80% train and 20% validation and will train a classifier. There will be 200 classifiers in total.

### 2. Train 200 teacher classifiers

In [26]:
num_teachers, num_examples, num_labels = 200, 300, 10

In [27]:
# define network architecture

class Teacher(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
    
    def forward(self, x):
        # flatten input tensor
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [29]:
# initialize 200 teacher classifiers

classifiers = []
for i in range(num_teachers):
    classifiers.append(Teacher())

In [37]:
# define some hyperparameters
batch_size = 64
epochs = 30
print_every = 5

In [82]:
# split teacher_data into 200 folds
kf = KFold(n_splits=num_teachers)

for i, (_, data_index) in enumerate(kf.split(teacher_data)):
    # take out one teacher subset
    teacher_subset = Subset(teacher_data, data_index)
    
    classifier = classifiers[i]
    
    print('Training teacher classifier: {}...'.format(i+1))
    
    # split teacher subset into 80% train and 20% test set
    num_samples = len(teacher_subset)
    indices = list(range(num_samples))
    np.random.shuffle(indices)
    split = int(np.floor(0.2 * num_samples))
    train_index, test_index = indices[split:], indices[:split]

    # define samplers for obtaining training and test batches
    train_sampler = SubsetRandomSampler(train_index)
    test_sampler = SubsetRandomSampler(test_index)

    # create dataloader
    trainloader = DataLoader(teacher_subset, batch_size=batch_size, sampler=train_sampler)
    testloader = DataLoader(teacher_subset, batch_size=batch_size, sampler=test_sampler)

    # specify loss and optimization function
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=0.003)

    # training
    classifier.train()

    for epoch in range(epochs):
        train_loss = 0
        for data, label in trainloader:
            optimizer.zero_grad()

            output = classifier(data)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)

        if (epoch+1) % print_every == 0:                 
            # switch to validation mode
            classifier.eval()
            test_loss = 0
            accuracy = 0
            with torch.no_grad():
                for data, label in testloader:
                    output = classifier(data)
                    loss = criterion(output, label)
                    test_loss += loss.item()*data.size(0)

                    # calculate accuracy
                    ps = torch.exp(output)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == label.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()


                print("Epoch: {}\tTraining loss: {:.4f}\tTest loss: {:.4f}\tTest accuracy: {:.4f}.".format(epoch+1,
                                                                                               train_loss/print_every,
                                                                                               test_loss/len(testloader),
                                                                                               accuracy/len(testloader)))
            # switch back to training mode
            classifier.train()

Training teacher classifier: 1...
Epoch: 5	Training loss: 1.4637	Test loss: 6.2646	Test accuracy: 0.9667.
Epoch: 10	Training loss: 0.2616	Test loss: 4.2905	Test accuracy: 0.9833.
Epoch: 15	Training loss: 0.0672	Test loss: 3.2211	Test accuracy: 0.9667.
Epoch: 20	Training loss: 0.0302	Test loss: 3.2978	Test accuracy: 0.9833.
Epoch: 25	Training loss: 0.0183	Test loss: 2.9324	Test accuracy: 0.9833.
Epoch: 30	Training loss: 0.0127	Test loss: 3.3265	Test accuracy: 0.9667.
Training teacher classifier: 2...
Epoch: 5	Training loss: 7.3138	Test loss: 9.2503	Test accuracy: 0.9500.
Epoch: 10	Training loss: 0.6189	Test loss: 7.5143	Test accuracy: 0.9667.
Epoch: 15	Training loss: 0.1156	Test loss: 4.5444	Test accuracy: 0.9667.
Epoch: 20	Training loss: 0.0518	Test loss: 7.3627	Test accuracy: 0.9667.
Epoch: 25	Training loss: 0.0345	Test loss: 6.6524	Test accuracy: 0.9500.
Epoch: 30	Training loss: 0.0257	Test loss: 6.7031	Test accuracy: 0.9500.
Training teacher classifier: 3...
Epoch: 5	Training loss: 

Epoch: 10	Training loss: 1.1709	Test loss: 17.6324	Test accuracy: 0.9500.
Epoch: 15	Training loss: 0.2185	Test loss: 20.7750	Test accuracy: 0.9500.
Epoch: 20	Training loss: 0.0961	Test loss: 20.6903	Test accuracy: 0.9500.
Epoch: 25	Training loss: 0.0616	Test loss: 22.0067	Test accuracy: 0.9500.
Epoch: 30	Training loss: 0.0452	Test loss: 22.6216	Test accuracy: 0.9500.
Training teacher classifier: 19...
Epoch: 5	Training loss: 7.8773	Test loss: 7.3084	Test accuracy: 0.9833.
Epoch: 10	Training loss: 0.9237	Test loss: 1.4568	Test accuracy: 1.0000.
Epoch: 15	Training loss: 0.2402	Test loss: 1.4116	Test accuracy: 0.9833.
Epoch: 20	Training loss: 0.1110	Test loss: 1.3534	Test accuracy: 0.9833.
Epoch: 25	Training loss: 0.0673	Test loss: 1.2135	Test accuracy: 0.9833.
Epoch: 30	Training loss: 0.0476	Test loss: 1.0414	Test accuracy: 1.0000.
Training teacher classifier: 20...
Epoch: 5	Training loss: 4.3273	Test loss: 5.8816	Test accuracy: 0.9833.
Epoch: 10	Training loss: 1.0861	Test loss: 5.3784	T

Epoch: 15	Training loss: 0.7312	Test loss: 21.0864	Test accuracy: 0.9167.
Epoch: 20	Training loss: 0.2153	Test loss: 22.5532	Test accuracy: 0.9167.
Epoch: 25	Training loss: 0.0989	Test loss: 23.0296	Test accuracy: 0.9333.
Epoch: 30	Training loss: 0.0627	Test loss: 23.9380	Test accuracy: 0.9333.
Training teacher classifier: 36...
Epoch: 5	Training loss: 6.5465	Test loss: 14.0605	Test accuracy: 0.9167.
Epoch: 10	Training loss: 1.0143	Test loss: 14.1654	Test accuracy: 0.9167.
Epoch: 15	Training loss: 0.2426	Test loss: 15.1061	Test accuracy: 0.9333.
Epoch: 20	Training loss: 0.0902	Test loss: 16.4725	Test accuracy: 0.9500.
Epoch: 25	Training loss: 0.0535	Test loss: 15.4297	Test accuracy: 0.9500.
Epoch: 30	Training loss: 0.0376	Test loss: 15.7455	Test accuracy: 0.9500.
Training teacher classifier: 37...
Epoch: 5	Training loss: 6.1660	Test loss: 8.6423	Test accuracy: 0.9500.
Epoch: 10	Training loss: 1.0852	Test loss: 9.2011	Test accuracy: 0.9500.
Epoch: 15	Training loss: 0.2502	Test loss: 9.4

Epoch: 20	Training loss: 0.6895	Test loss: 4.7267	Test accuracy: 0.9667.
Epoch: 25	Training loss: 0.3708	Test loss: 4.5918	Test accuracy: 0.9833.
Epoch: 30	Training loss: 0.2448	Test loss: 4.6719	Test accuracy: 0.9833.
Training teacher classifier: 53...
Epoch: 5	Training loss: 9.2495	Test loss: 22.3234	Test accuracy: 0.8667.
Epoch: 10	Training loss: 1.7827	Test loss: 12.8248	Test accuracy: 0.9167.
Epoch: 15	Training loss: 0.3501	Test loss: 12.8443	Test accuracy: 0.9333.
Epoch: 20	Training loss: 0.1402	Test loss: 12.4270	Test accuracy: 0.9333.
Epoch: 25	Training loss: 0.0884	Test loss: 13.1646	Test accuracy: 0.9333.
Epoch: 30	Training loss: 0.0652	Test loss: 13.2690	Test accuracy: 0.9333.
Training teacher classifier: 54...
Epoch: 5	Training loss: 11.2648	Test loss: 15.6869	Test accuracy: 0.9000.
Epoch: 10	Training loss: 1.9733	Test loss: 10.0446	Test accuracy: 0.9333.
Epoch: 15	Training loss: 0.2478	Test loss: 10.7571	Test accuracy: 0.9333.
Epoch: 20	Training loss: 0.1049	Test loss: 10.

Epoch: 25	Training loss: 0.0179	Test loss: 4.9166	Test accuracy: 0.9667.
Epoch: 30	Training loss: 0.0138	Test loss: 4.7876	Test accuracy: 0.9667.
Training teacher classifier: 70...
Epoch: 5	Training loss: 9.5698	Test loss: 23.5632	Test accuracy: 0.9000.
Epoch: 10	Training loss: 1.6505	Test loss: 23.2703	Test accuracy: 0.9500.
Epoch: 15	Training loss: 0.3020	Test loss: 22.5699	Test accuracy: 0.9500.
Epoch: 20	Training loss: 0.1428	Test loss: 25.8509	Test accuracy: 0.9500.
Epoch: 25	Training loss: 0.0910	Test loss: 25.0815	Test accuracy: 0.9500.
Epoch: 30	Training loss: 0.0696	Test loss: 25.5178	Test accuracy: 0.9500.
Training teacher classifier: 71...
Epoch: 5	Training loss: 13.8041	Test loss: 13.0554	Test accuracy: 0.9500.
Epoch: 10	Training loss: 2.1703	Test loss: 10.8837	Test accuracy: 0.9333.
Epoch: 15	Training loss: 0.4793	Test loss: 17.0968	Test accuracy: 0.9500.
Epoch: 20	Training loss: 0.1738	Test loss: 16.9329	Test accuracy: 0.9333.
Epoch: 25	Training loss: 0.0957	Test loss: 16

...

Epoch: 5	Training loss: 14.2919	Test loss: 15.2170	Test accuracy: 0.9500.
Epoch: 10	Training loss: 2.8901	Test loss: 7.3007	Test accuracy: 0.9500.
Epoch: 15	Training loss: 0.5011	Test loss: 5.1167	Test accuracy: 0.9833.
Epoch: 20	Training loss: 0.1937	Test loss: 3.4068	Test accuracy: 0.9667.
Epoch: 25	Training loss: 0.1095	Test loss: 3.6710	Test accuracy: 0.9667.
Epoch: 30	Training loss: 0.0756	Test loss: 3.4457	Test accuracy: 0.9833.
Training teacher classifier: 122...
Epoch: 5	Training loss: 5.6286	Test loss: 16.2397	Test accuracy: 0.9333.
Epoch: 10	Training loss: 0.8724	Test loss: 10.9656	Test accuracy: 0.9667.
Epoch: 15	Training loss: 0.1925	Test loss: 11.7718	Test accuracy: 0.9500.
Epoch: 20	Training loss: 0.0841	Test loss: 11.7774	Test accuracy: 0.9500.
Epoch: 25	Training loss: 0.0514	Test loss: 12.2676	Test accuracy: 0.9667.
Epoch: 30	Training loss: 0.0372	Test loss: 12.4257	Test accuracy: 0.9667.
Training teacher classifier: 123...
Epoch: 5	Training loss: 4.5805	Test loss: 22.1

Epoch: 10	Training loss: 0.8961	Test loss: 3.4341	Test accuracy: 0.9833.
Epoch: 15	Training loss: 0.1643	Test loss: 1.3217	Test accuracy: 1.0000.
Epoch: 20	Training loss: 0.0745	Test loss: 1.1852	Test accuracy: 1.0000.
Epoch: 25	Training loss: 0.0466	Test loss: 1.4365	Test accuracy: 1.0000.
Epoch: 30	Training loss: 0.0333	Test loss: 1.5510	Test accuracy: 0.9833.
Training teacher classifier: 139...
Epoch: 5	Training loss: 6.5374	Test loss: 14.1505	Test accuracy: 0.9167.
Epoch: 10	Training loss: 0.7857	Test loss: 9.3861	Test accuracy: 0.9667.
Epoch: 15	Training loss: 0.1628	Test loss: 9.4288	Test accuracy: 0.9667.
Epoch: 20	Training loss: 0.0830	Test loss: 10.0445	Test accuracy: 0.9667.
Epoch: 25	Training loss: 0.0568	Test loss: 10.8221	Test accuracy: 0.9667.
Epoch: 30	Training loss: 0.0434	Test loss: 11.3559	Test accuracy: 0.9667.
Training teacher classifier: 140...
Epoch: 5	Training loss: 10.4442	Test loss: 9.1820	Test accuracy: 0.9833.
Epoch: 10	Training loss: 2.4230	Test loss: 11.788

Epoch: 15	Training loss: 0.3819	Test loss: 14.9151	Test accuracy: 0.9500.
Epoch: 20	Training loss: 0.1735	Test loss: 16.3472	Test accuracy: 0.9333.
Epoch: 25	Training loss: 0.1108	Test loss: 15.5769	Test accuracy: 0.9333.
Epoch: 30	Training loss: 0.0800	Test loss: 16.4103	Test accuracy: 0.9333.
Training teacher classifier: 156...
Epoch: 5	Training loss: 3.6278	Test loss: 18.9609	Test accuracy: 0.9167.
Epoch: 10	Training loss: 0.3888	Test loss: 14.9094	Test accuracy: 0.8833.
Epoch: 15	Training loss: 0.0964	Test loss: 12.6614	Test accuracy: 0.9167.
Epoch: 20	Training loss: 0.0491	Test loss: 13.8477	Test accuracy: 0.9333.
Epoch: 25	Training loss: 0.0323	Test loss: 15.0256	Test accuracy: 0.9167.
Epoch: 30	Training loss: 0.0239	Test loss: 15.7619	Test accuracy: 0.9167.
Training teacher classifier: 157...
Epoch: 5	Training loss: 6.9885	Test loss: 14.8151	Test accuracy: 0.9167.
Epoch: 10	Training loss: 1.8867	Test loss: 9.3997	Test accuracy: 0.9333.
Epoch: 15	Training loss: 0.3540	Test loss: 

Epoch: 15	Training loss: 0.9173	Test loss: 3.7380	Test accuracy: 0.9833.
Epoch: 20	Training loss: 0.4246	Test loss: 3.7811	Test accuracy: 0.9833.
Epoch: 25	Training loss: 0.1363	Test loss: 3.4752	Test accuracy: 0.9833.
Epoch: 30	Training loss: 0.0806	Test loss: 3.2720	Test accuracy: 0.9833.
Training teacher classifier: 173...
Epoch: 5	Training loss: 8.3330	Test loss: 14.6072	Test accuracy: 0.9500.
Epoch: 10	Training loss: 2.2860	Test loss: 9.8172	Test accuracy: 0.9667.
Epoch: 15	Training loss: 0.5252	Test loss: 8.8126	Test accuracy: 0.9667.
Epoch: 20	Training loss: 0.2149	Test loss: 9.1337	Test accuracy: 0.9667.
Epoch: 25	Training loss: 0.1285	Test loss: 9.9588	Test accuracy: 0.9667.
Epoch: 30	Training loss: 0.0901	Test loss: 10.3481	Test accuracy: 0.9667.
Training teacher classifier: 174...
Epoch: 5	Training loss: 11.8097	Test loss: 21.5702	Test accuracy: 0.9500.
Epoch: 10	Training loss: 1.7765	Test loss: 14.3420	Test accuracy: 0.9500.
Epoch: 15	Training loss: 0.2894	Test loss: 11.022

Epoch: 15	Training loss: 0.5447	Test loss: 10.6049	Test accuracy: 0.9833.
Epoch: 20	Training loss: 0.1431	Test loss: 11.4846	Test accuracy: 0.9500.
Epoch: 25	Training loss: 0.0699	Test loss: 13.2333	Test accuracy: 0.9333.
Epoch: 30	Training loss: 0.0441	Test loss: 13.8643	Test accuracy: 0.9167.
Training teacher classifier: 190...
Epoch: 5	Training loss: 7.3467	Test loss: 15.7145	Test accuracy: 0.8833.
Epoch: 10	Training loss: 0.6182	Test loss: 1.4758	Test accuracy: 1.0000.
Epoch: 15	Training loss: 0.1433	Test loss: 1.5123	Test accuracy: 1.0000.
Epoch: 20	Training loss: 0.0728	Test loss: 1.6599	Test accuracy: 0.9833.
Epoch: 25	Training loss: 0.0535	Test loss: 1.5246	Test accuracy: 0.9833.
Epoch: 30	Training loss: 0.0421	Test loss: 1.4647	Test accuracy: 0.9833.
Training teacher classifier: 191...
Epoch: 5	Training loss: 10.5287	Test loss: 18.9316	Test accuracy: 0.9500.
Epoch: 10	Training loss: 1.4602	Test loss: 15.3205	Test accuracy: 0.9667.
Epoch: 15	Training loss: 0.3399	Test loss: 15.

In [83]:
# save 200 classifiers

all_points = []

for i in range(len(classifiers)):
    all_points.append(classifiers[i].state_dict())
    
checkpoint = {'classifiers_list': all_points}
torch.save(checkpoint, 'checkpoints/teachers_checkpoint.pth')

In [84]:
# load checkpoint

checkpoint = torch.load('checkpoints/teachers_checkpoint.pth')
all_points = checkpoint['classifiers_list']

for i in range(len(all_points)):
    classifiers[i].load_state_dict(all_points[i])

1. Devide teacher dataset (60000 samples) into 200 different teacher subsets, each subset contains 300 samples.
2. For each subset, train a classifier. There will be 200 classifiers in total.
3. In each subset, split into 80% train and 20% validation.
4. Save 300 classifiers then load them.
5. Use these classifiers to predict the labels for test_data (or student_data), but the training part only.
6. Perform PATE analysis
7. Build a model on student dataset. Split student dataset into 80% train and 20% test (maybe validation as well).
8. Train this model with created labels.
9. Calculate accuracy on test set, using ground truth labels.